In [7]:
import pandas as pd
from datetime import datetime
from datetime import timezone
from datetime import timedelta
import numpy as np
from ib_insync import *

In [2]:
util.startLoop()
ib = IB()
ib.connect('127.0.0.1', 7496, clientId=15)

<IB connected to 127.0.0.1:7496 clientId=15>

In [57]:
def process_data(tick):
    df = pd.DataFrame(tick)
    dftime_local = df['time']
    df['time'] = pd.to_datetime(dftime_local, '%Y-%m-%d %H:%M:%S').astype(str).map(lambda x: str(x)[0:19])
    
    long = len(df)
    avg_price = (df.priceBid + df.priceAsk)/2
    ttl_volume = df.sizeBid + df.sizeAsk

    new_df = pd.DataFrame(np.random.randn(long,3),columns=['time','price','volume'])
    new_df['time'] = df.time
    new_df['price'] = avg_price
    new_df['volume'] = ttl_volume
    return new_df

In [58]:
def data_convert(ticks):
    df = pd.DataFrame(index =ticks.time.values)
    df['price'] = ticks['price'].values
    df['volume'] = ticks['volume'].values
    return df
    

In [72]:
def check_excess(start,ticks,end):
    
    ticks = ticks[pd.to_datetime(ticks.index,utc=True)<=pd.to_datetime(end,utc=True)]
    ticks = ticks[pd.to_datetime(ticks.index,utc=True)>=pd.to_datetime(start,utc=True)]
    return ticks

In [75]:
def utc_to_local(utc_dt):
    return utc_dt.replace(tzinfo=timezone.utc).astimezone(tz=None)
def his_tick_extract(start,end,contracts):
    #start: start time for tick data
    #end: end time for tick data
    ib.qualifyContracts(*contracts)
    amd = contracts[0]
    temp_start = start
    ticks=pd.DataFrame(columns = ['time','price','volume'])
    while  pd.to_datetime(temp_start,utc=True) <= pd.to_datetime(end,utc=True):
        temp_end = ''
        tick = ib.reqHistoricalTicks(amd, temp_start, temp_end, 1000, 'BID_ASK', useRth=False)
        temp_start = tick[-1][0]
        print("Tick data length",len(tick))
        if len(tick)<1000:
            print('Jump at this time')
            temp_start += timedelta(seconds=1)
        print(temp_start)
        tick = pd.DataFrame(tick)
        tick = process_data(tick)
        tick['count'] = tick.groupby(['time','price','volume']).cumcount()
        ticks['count'] = ticks.groupby(['time','price','volume']).cumcount()

        ticks = pd.concat([ticks,tick], ignore_index=False)
        ticks = ticks.drop_duplicates()
        ticks = ticks.drop(['count'], axis=1)
    ticks = data_convert(ticks)
    ticks = check_excess(start,ticks,end)
    return ticks

In [76]:
start = datetime(2019, 7, 2, 0, 0, 0, 0, tzinfo=timezone.utc)
end = datetime(2019, 7, 2, 0, 10, 0, 0, tzinfo=timezone.utc)
contracts = [Forex(pair) for pair in ('EURUSD', 'USDJPY', 'GBPUSD', 'USDCHF', 'USDCAD', 'AUDUSD')]
tick1=his_tick_extract(start,end,contracts)

Tick data length 1017
2019-07-02 00:05:02+00:00
Tick data length 1016
2019-07-02 00:07:35+00:00
Tick data length 1021
2019-07-02 00:11:03+00:00


In [77]:
tick1

,price,volume
2019-07-02 00:00:00,1.128695,3000000
2019-07-02 00:00:00,1.128695,4000000
2019-07-02 00:00:01,1.128695,4000000
2019-07-02 00:00:01,1.128695,5000000
2019-07-02 00:00:01,1.128695,5000000
2019-07-02 00:00:01,1.128695,6000000
2019-07-02 00:00:01,1.128700,7000000
2019-07-02 00:00:01,1.128700,6000000
2019-07-02 00:00:01,1.128700,7000000
2019-07-02 00:00:01,1.128700,10000000


In [55]:
#tick1['time'] = tick1.index
t1 = tick1.iloc[:1017,:]
t2 = tick1.iloc[1017:,:]
t1['1']=1

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [56]:
t1['count'] = t1.groupby(['time','price','volume']).cumcount()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [40]:
t1['count'] = t1.groupby(['time','price','volume']).cumcount()
t2['count'] = t2.groupby(['time','price','volume']).cumcount()

df_tot = pd.concat([t1,t2], ignore_index=False)
df_tot = df_tot.drop_duplicates()
df_tot = df_tot.drop(['count'], axis=1)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [43]:
df_tot.iloc[:1017,:]
df_tot.iloc[1017:,:]

,time,price,volume
1055,2019-07-02 00:05:03,1.128730,8000000
1056,2019-07-02 00:05:03,1.128730,8500000
1057,2019-07-02 00:05:03,1.128730,7500000
1058,2019-07-02 00:05:03,1.128730,6500000
1059,2019-07-02 00:05:03,1.128730,5500000
1060,2019-07-02 00:05:05,1.128730,4500000
1061,2019-07-02 00:05:05,1.128730,5500000
1062,2019-07-02 00:05:05,1.128730,7500000
1063,2019-07-02 00:05:05,1.128730,8500000
1064,2019-07-02 00:05:05,1.128730,7500000
